In [2]:
import torch

ModuleNotFoundError: No module named 'torch'

In [1]:
words = open('words.txt', 'r').read().splitlines()

In [ ]:
chars = sorted(list(set(''.join(['.'] + words))))

stoi = {s:i for s, i in enumerate(chars)}
itos = {i:s for s, i in stoi.items()}

In [ ]:
xs,  ys = [], []

for word in words[:1]:
  chs = ['.'] * 2 + list(word) + ['.'] * 2
  for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
    ix1 = itos[ch1]
    ix2 = itos[ch2]
    ix3 = itos[ch3]
    xs.append([ix1, ix2])
    ys.append(ix3)

xs = torch.tensor(xs)
ys = torch.tensor(ys)

In [ ]:
P = (N+1).float()
P /= P.sum(2, keepdim=True)

In [ ]:
g = torch.Generator().manual_seed(2147483647)

for i in range(5):
  out = []
  ix1 = 0
  ix2 = 0
  while True:
    p = P[ix1, ix2]
    ix_new = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
    out.append(stoi[ix_new])
    ix1 = ix2
    ix2 = ix_new
    if ix_new == 0:
      break
  print(''.join(out))


In [ ]:
log_likelihood = 0.0
n = 0

for word in words:
  chs = ['.'] * 2 + list(word) + ['.'] * 2
  for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
    ix1 = itos[ch1]
    ix2 = itos[ch2]
    ix3 = itos[ch3]
    prob = P[ix1, ix2, ix3]
    log_likelihood += torch.log(prob)
    n += 1
nll = -log_likelihood
print(f'{nll=:.2f}')
print(f'{nll/n:.2f}')